In [260]:
%matplotlib qt
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm

In [261]:
# Load image


def subsample_image(image, k=0.25):
    return cv2.resize(image, None, fx=k, fy=k, 
                      interpolation=cv2.INTER_NEAREST)


imbgr_rect = cv2.imread(
    '/home/chao/Workspace/repo/kumar_robotics/Galt/detection/labeled_data/frame0001.png')
imgray_apple = cv2.imread(
    '/home/chao/Workspace/repo/kumar_robotics/Galt/detection/labeled_data/frame0001_apple.png',
    cv2.IMREAD_GRAYSCALE)
imgray_other = cv2.imread(
    '/home/chao/Workspace/repo/kumar_robotics/Galt/detection/labeled_data/frame0001_other.png',
    cv2.IMREAD_GRAYSCALE)

# Resize image to half
imgray_apple = subsample_image(imgray_apple)
imgray_other = subsample_image(imgray_other)
imbgr_rect = subsample_image(imbgr_rect)

In [262]:
# Extract bgr training data from image
bw_apple = imgray_apple > 0
imbgr_apple = np.array(imbgr_rect, copy=True)
imbgr_apple[~bw_apple] = 0

x_apple_bgr = imbgr_rect[bw_apple]
n_x_apple = len(x_apple_bgr)
print('apple: ', n_x_apple)
plt.imshow(imbgr_apple)

('apple: ', 1195)


In [263]:
bw_other = imgray_other > 0
imbgr_other = np.array(imbgr_rect, copy=True)
imbgr_other[~bw_other] = 0
x_other_bgr = imbgr_rect[bw_other]

# Usually a lot more other than apple, so we subsample other to 
# be approximately the same as apple
r_other_apple = int(len(x_other_bgr) / n_x_apple / 2)
x_other_bgr = x_other_bgr[::r_other_apple]
n_x_other = len(x_other_bgr)
print('other: ', n_x_other)
plt.imshow(imbgr_other)

('other: ', 2589)


In [265]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
r = 2
x_apple_bgr_sub = x_apple_bgr[::r]
x_other_bgr_sub = x_other_bgr[::r]
ax.scatter(x_apple_bgr_sub[:, 0], x_apple_bgr_sub[:, 1], x_apple_bgr_sub[:, 2], c='r')
ax.scatter(x_other_bgr_sub[:, 0], x_other_bgr_sub[:, 1], x_other_bgr_sub[:, 2], c='b')
ax.set_xlabel('b')
ax.set_ylabel('g')
ax.set_zlabel('r')
plt.show()

In [266]:
# Test with hsv colorspace

imhsv_rect = cv2.cvtColor(imbgr_rect, cv2.COLOR_BGR2HSV)

In [267]:
# Extract hsv for apple and other
x_apple_hsv = imhsv_rect[bw_apple]
x_other_hsv = imhsv_rect[bw_other]
x_other_hsv = x_other_hsv[::r_other_apple]

In [268]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
r = 2
x_apple_hsv_sub = x_apple_hsv[::r]
x_other_hsv_sub = x_other_hsv[::r]
ax.scatter(x_apple_hsv_sub[:, 0], x_apple_hsv_sub[:, 1], x_apple_hsv_sub[:, 2], c='r')
ax.scatter(x_other_hsv_sub[:, 0], x_other_hsv_sub[:, 1], x_other_hsv_sub[:, 2], c='b')
ax.set_xlabel('b')
ax.set_ylabel('g')
ax.set_zlabel('r')
plt.show()

In [269]:
# Test with LAB colorspace
imlab_rect = cv2.cvtColor(imbgr_rect, cv2.COLOR_BGR2LAB)
x_apple_lab = imlab_rect[bw_apple]
x_other_lab = imlab_rect[bw_other]
x_other_lab = x_other_lab[::r_other_apple]

In [270]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
r = 2
x_apple_lab_sub = x_apple_lab[::r]
x_other_lab_sub = x_other_lab[::r]
ax.scatter(x_apple_lab_sub[:, 0], x_apple_lab_sub[:, 1], x_apple_lab_sub[:, 2], c='r')
ax.scatter(x_other_lab_sub[:, 0], x_other_lab_sub[:, 1], x_other_lab_sub[:, 2], c='b')
ax.set_xlabel('b')
ax.set_ylabel('g')
ax.set_zlabel('r')
plt.show()

In [271]:
print(x_apple_hsv.shape)
print(x_apple_bgr.shape)
print(x_apple_lab.shape)
print(x_other_hsv.shape)
print(x_other_bgr.shape)
print(x_other_lab.shape)

(1195, 3)
(1195, 3)
(1195, 3)
(2589, 3)
(2589, 3)
(2589, 3)


In [272]:
# Make a big feature vector
r = 1
x_apple = np.hstack((x_apple_bgr[::r], x_apple_hsv[::r], x_apple_lab[::r]))
x_other = np.hstack((x_other_bgr[::r], x_other_hsv[::r], x_other_lab[::r]))
x_train = np.vstack((x_apple, x_other))

y_apple = np.ones((np.size(x_apple, axis=0),))
y_other = np.zeros((np.size(x_other, axis=0),))
y_train = np.hstack((y_apple, y_other))

print('x_train: ', x_train.shape)
print('y_train: ', y_train.shape)

('x_train: ', (3784, 9))
('y_train: ', (3784,))


In [273]:
# Use SVM for classification
clf = svm.SVC()
clf.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [277]:
# Load another photo for testing
imbgr_rect_test = cv2.imread(
    '/home/chao/Workspace/repo/kumar_robotics/Galt/detection/labeled_data/frame0000.png')
imbgr_rect_test = subsample_image(imbgr_rect_test)
imhsv_rect_test = cv2.cvtColor(imbgr_rect_test, cv2.COLOR_BGR2HSV)
imlab_rect_test = cv2.cvtColor(imbgr_rect_test, cv2.COLOR_BGR2LAB)

In [275]:
# Create a big feature vector for testing
h, w, c = np.shape(imbgr_rect_test)
x_test_bgr = np.reshape(imbgr_rect_test, (h * w, -1))
x_test_hsv = np.reshape(imhsv_rect_test, (h * w, -1))
x_test_lab = np.reshape(imlab_rect_test, (h * w, -1))
x_test = np.hstack((x_test_bgr, x_test_hsv, x_test_lab))
print('x_test: ', x_test.shape)

('x_test: ', (144000, 9))


In [276]:
y_test = clf.predict(x_test)

In [278]:
bw_y_test = np.reshape(y_test, (h , w))
bw_y_test = bw_y_test > 0
plt.imshow(bw_y_test)

In [279]:
imbgr_test_apple = np.array(imbgr_rect_test, copy=True)
imbgr_test_apple[~bw_y_test, :] = 255
plt.imshow(imbgr_test_apple)